In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

# An introduction to Gradual Typing in Python

Gradual typing is a programming language feature that allows developers to gradually add type annotations to their code. In Python, this means you can mix statically typed and dynamically typed code within the same program, taking advantage of both worlds. Type annotations can be added to variables, function parameters, and function return values, offering better documentation and enabling more robust static analysis through tools like [mypy](https://mypy-lang.org/). The type system is designed to be flexible and can accommodate a wide range of patterns, including 
+ *generic types*, 
+ *union types*, and 
+ *user defined types*. 

By incrementally adding type annotations, developers can make their Python code more self-explanatory, easier to debug, and more maintainable, while also catching potential type errors before runtime. 

In order to use `mypy` in a Jupyter notebook, we first have to load the notebook extension for `mypy`.

In [ ]:
%load_ext nb_mypy

## Finding Errors by Type Checking

The following two lines contain an error that `mypy` is able to find.

In [ ]:
number = input("What is your favourite number? ")
print("It is", number + 1)  

The correct version of the two lines above would have been as follows:

In [ ]:
number = int(input("What is your favourite number? "))
print(f'It is {number + 1}.') 

## Type Annotations

The most basic form of type checking in Python is specifying the types of variables and function return values.
To type check a function, you annotate the type of a parameter by putting a colon after the name of the variable. 
The return type of the function is specified using the `->` syntax as shown below.

In [ ]:
def add(a: int, b: int) -> int:
    return a + b

In the next cell the **type checker** tells us, that we have called the function `add` with strings instead of integers.  
The **Python interpreter** executes this cell without encountering an error, since the interpreter does not care about the type annotations. 

In [ ]:
name = 'Karl'
add('Hello ', name)    

If necessary, we can inspect the type annotations of a function at runtime via the attribute `__annotations__` as shown below.

In [ ]:
add.__annotations__

If we do not use type annotations for a function `f`, the dictionary `f.__annotations__` is empty.

In [ ]:
def sub(a, b):
    return a - b

In [ ]:
sub.__annotations__

## Built-in Types

`mypy` supports all built-in Python types like `int`, `float`, `str`, and `bool`.
Complex types like `list`, `tuple`, and `dict` are also supported.  

The function `average(L)` computes the arithmetic mean of the numbers in the list `L`. 

In [ ]:
def average(numbers: list[int|float]) -> float:
    return sum(numbers) / len(numbers)

In [ ]:
average([1, 2, 3, 4])

## Custom Types
You can define your own types using the `class` keyword. Note that the parameter `self` should not have a type annotation.
The reason is that `self` implicitly has the type `Person`.

In [ ]:
class Person:
    def __init__(self, name: str):
        self.name = name

    def greet(self) -> str:
        return f"Hello, {self.name}!"

When a function does not return a value, the return type is `None`.

In [ ]:
def salve(p: Person) -> None:
    print(p.greet())

In [ ]:
jc = Person('Julius Caesar')
salve(jc)

The function `greet_name` either accepts a string representing a name as its argument, 
or it accepts a dictionary as its argument.  The dictionary is supposed to store both 
the first name under the key `given` and the last name under the key `family`.  

The *union* operator `|` can be used to express the fact that `name`can either be a
`str` or a `dict[str, str]`.

In [ ]:
def greet_name(name: str | dict[str, str]) -> str:
    if isinstance(name, str):
        return 'Hi ' + name + '!'
    if isinstance(name, dict):
        return f"Bienvenido, Señor {name['given']} {name['family']}."

In [ ]:
greet_name("Alice")

In [ ]:
greet_name({'given': 'Esteban', 'family': 'Ramirez'})

## Typing Generic Functions Using `TypeVar`

In [ ]:
from typing import TypeVar

The next example shows how to type *generic*  functions.  This is done using the function `TypeVar`, 
which creates a new type variable.

In [ ]:
S = TypeVar('S')
T = TypeVar('T')

The function `swap` takes a pair of elements that should be of the same type.  
It swaps the order of these elements. swaps the elements of a pair (a 2-tuple). The function   
`swap` is *generic*, meaning it is able to handle pairs of integers, strings, or any other type.

In [ ]:
def swap(pair: tuple[S, T]) -> tuple[T, S]:
    x, y = pair
    return y, x

In the next cell, the type variable `T` is instantiated as `int`.

In [ ]:
swap((1, 2))

In the following cell, the type variable `T` is instantiated as `str`.

In [ ]:
swap(('a', 'b'))

Below, the type variable the type variable `T` is instantiated as `object`.

In [ ]:
swap((1, 'a'))

## Typing Generic Functions Using Square Brackets

In [ ]:
def rotate[U,V,W](triple: tuple[U, V, W]) -> tuple[V, W, U]:
    x, y, z = triple
    return y, z, x

In [ ]:
rotate( (1, 'a', True) )

## Recursive Types

In [ ]:
from typing import Literal

In [ ]:
type Operator = Literal['+', '-', '*', '/']

In [ ]:
type Expression = float | str | tuple[Expression, Operator, Expression] 

The function `differentiate(e, x)` takes an arithmetic expression `e` and a variable `x`
and compute the derivative of `e` with respect to `x`.

In [ ]:
def differentiate(expr: Expression, var: str = 'x') -> Expression:
    match expr:
        case float():
            return 0.0
        case str():
            return 1.0 if expr == var else 0.0
        case (lhs, op, rhs):
            dLhs = differentiate(lhs, var)
            dRhs = differentiate(rhs, var)
            match op:
                case '+':
                    return (dLhs, '+', dRhs)
                case '-':
                    return (dLhs, '-', dRhs)
                case '*':
                    p1: Expression = (dLhs, '*',  rhs)
                    p2: Expression = ( lhs, '*', dRhs)    
                    return (p1, '+', p2)
                case '/':
                    q1: Expression = (dLhs, '*',  rhs)
                    q2: Expression = ( lhs, '*', dRhs)    
                    numerator   : Expression = (q1, '-', q2)
                    denominator : Expression = (rhs, '*', rhs)
                    return (numerator, '/', denominator)
    return 0.0

Unfortunately, due to an error in mypy, the function below does not type check.

In [ ]:
def differentiate(expr: Expression, var: str = 'x') -> Expression:
    match expr:
        case float():
            return 0.0
        case str():
            return 1.0 if expr == var else 0.0
        case (lhs, op, rhs):
            dLhs = differentiate(lhs, var)
            dRhs = differentiate(rhs, var)
            match op:
                case '+':
                    return (dLhs, '+', dRhs)
                case '-':
                    return (dLhs, '-', dRhs)
                case '*':
                    return ((dLhs, '*', dRhs), '+', (lhs, '*', dRhs))
                case '/':
                    numerator   = ((dLhs, '*', rhs), '-', (lhs, '*', dRhs))
                    denominator = (rhs, '*', rhs)
                    return (numerator, '/', denominator)
    return 0.0